In [1]:
import random
import re
import requests
import time
import math
import numpy as np
import operator
from bs4 import BeautifulSoup as bs
from collections import Counter
import os

In [17]:
import requests
import re
import time
from bs4 import BeautifulSoup as bs

def removeTag(soup,tag):
    [x.extract() for x in soup.select(tag)]

def geturl(f, p, rs):
    urls = []
    domain = 'http://www.mobile01.com/'
    url = 'http://www.mobile01.com/waypointtopiclist.php?f={}&p={}'
    #url = 'http://www.mobile01.com/waypointforumtopic.php?p={}'
    res = rs.get(url.format(f,p))
    soup = bs(res.text,'html.parser')
    page = int(re.search('共(.+)頁', soup.select('.numbers')[0].encode('utf-8')).group(1))
    titles = soup.select('.subject-text')
    for t in titles:
        urls.append(domain+t.select('a')[0]['href'])
    return urls

def getFinalPage(url, rs):
    res = rs.get(url)
    soup = bs(res.text,'lxml')
    page = int(re.search('共(.+)頁', soup.select('.numbers')[0].encode('utf-8')).group(1))
    return page

def getcontent(url, rs):
    res = rs.get(url)
    soup = bs(res.text,'html.parser')
    removeTag(soup,'script')
    removeTag(soup,'a')
    removeTag(soup,'.rank')
    removeTag(soup,'iframe')
    removeTag(soup,'.fsb-social-bar')
    removeTag(soup,'small')
    removeTag(soup,'.comment-content.comment')
    removeTag(soup,'.moreincategories.clearfix')
    removeTag(soup,'.relativepost.clearfix')
    removeTag(soup,'.auth')
    removeTag(soup,'.store')
    removeTag(soup,'.comments-area')
    removeTag(soup,'.sharedaddy.sd-sharing-enabled')
    removeTag(soup,'.vcard')
    removeTag(soup,'#facebook')
    removeTag(soup,'#sidebar')
    removeTag(soup,'#jp-relatedposts')
    removeTag(soup,'.hot-info')
    removeTag(soup,'.agoda-link')
    removeTag(soup,'#notice')
    removeTag(soup,'.tag')
    removeTag(soup,'blockquote')
    t = soup.select('.single-post-content')[0]
    s =  "".join("".join(t.text).split())
    return s

def getall(f, p):
    st = []
    ul = geturl(f, p, rs)
    for u in ul:
        try:
            st.append(getcontent(u, rs))
        except:
            print u
        time.sleep(0.5)
    return st

In [24]:
#import autoDict
# 一整頁文章一起跑
#頁數範圍
for f in xrange(196,213):
    url = 'http://www.mobile01.com/waypointtopiclist.php?f={}'
    try:
        fp = getFinalPage(url.format(f), rs)
    except IndexError:
        continue
    for p in xrange(1,fp+1):
        with open('./Workspace/data/mobile01/article_%s_%d.txt' %(f,p), 'w') as fw:
            #獲取一整頁的文章
            news = getall(f, p)
            for a in news:
                fw.write(a.encode('utf-8')+'\n')
            #autoDict.getwords(news)

In [5]:
import codecs
temp = set()
for fname in os.listdir('./Workspace/data/dictionary'):
    if 'txt' in fname:
        with codecs.open('./Workspace/data/dictionary/%s' %fname, 'r', 'utf-8') as f:
            for t in f.readlines():
                if not re.search('\w', t) and u'的' not in t:
                    temp.add(t.strip('\n'))
with open('./Workspace/data/dictionary/all/dictionary.txt', 'w') as f:
    for x in list(temp):
        f.write(x.encode('utf-8')+'\n')
print len(temp)

6499
